# Knowledge Graph Querying: Competency Questions (CQs)
This notebook shows how to query KGs to answer the CQs related to the OntoBOT ontology.

We will:
- load local RDF graphs
- Formulate SPARQL queries corresponding to CQ

#### 🔧 Setup 🔧

In [1]:
from rdflib import Graph, Namespace
from collections import defaultdict
from rdflib.namespace import RDFS, FOAF, DCTERMS

import pandas as pd

In [25]:
# Load graph
g = Graph()
g.parse("scenarios_kgs.ttl", format="ttl")

<Graph identifier=N836aa0c5daa94b488d254b2caf312511 (<class 'rdflib.graph.Graph'>)>

In [ ]:
HI = Namespace("https://w3id.org/hi-ontology#")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
HI_S1 = Namespace("https://w3id.org/hi-ontology/scenarios/1")
HI_S2 = Namespace("https://w3id.org/hi-ontology/scenarios/2")
HI_S3 = Namespace("https://w3id.org/hi-ontology/scenarios/3")
HI_S4 = Namespace("https://w3id.org/hi-ontology/scenarios/4")
HI_S5 = Namespace("https://w3id.org/hi-ontology/scenarios/5")
HI_S6 = Namespace("https://w3id.org/hi-ontology/scenarios/6")
HI_S7 = Namespace("https://w3id.org/hi-ontology/scenarios/7")

g.bind("hi", HI)
g.bind("dul", DUL)
g.bind("hi-s", HI_S1)
g.bind("hi-s", HI_S2)
g.bind("hi-s", HI_S3)
g.bind("hi-s", HI_S4)
g.bind("hi-s", HI_S5)
g.bind("hi-s", HI_S6)
g.bind("hi-s", HI_S7)

#### 🤖 CQ 1 🤖

**Can we identify, within a given Hybrid Intelligence scenario, the participants involved in the human–AI collaboration and the roles they assume?**

In [ ]:
cq1 = """
PREFIX hi: <https://w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?participantLabel ?roleLabel
WHERE {
    VALUES ?scenario { hi-s1:HIScenarioPersonalAssistant }
    
    ?scenario       hi:describesUseCase     ?usecase .
    ?usecase        hi:introducesHITeam     ?team .
    ?team           hi:hasMember            ?participant .
    ?participant    hi:hasRole              ?role ;
                    rdfs:label              ?participantLabel .
    ?role           rdfs:label              ?roleLabel .
}
"""

results_cq1 = g.query(cq1)
df_cq1 = pd.DataFrame([(str(row.participantLabel), str(row.roleLabel)) for row in results_cq1],
                        columns=["Participant", "Role"])
df_cq1

,Participant,Role
0,AI Assistant,Personal Assistant System Role.
1,Human User,Human User Role.


#### 🤖 CQ 2 🤖

**What are the capabilities required by each participant to perform a specific task within a Hybrid Intelligence goal?**

In [ ]:
cq2 = """
PREFIX hi: <https://w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?participantLabel ?capabilityLabel
WHERE {
    VALUES ?scenario { hi-s1:HIScenarioPersonalAssistant }
    
    ?scenario       hi:describesUseCase     ?usecase .
    ?usecase        hi:introducesHITeam     ?team .
    ?team           hi:hasMember            ?participant .
    ?participant    rdfs:label              ?participantLabel ;
                    hi:hasCapability        ?capability .
    ?capability     rdfs:label              ?capabilityLabel .
}
"""

results_cq2 = g.query(cq2)
df_cq2 = pd.DataFrame([(str(row.participantLabel), str(row.capabilityLabel)) for row in results_cq2],
                        columns=["Participant", "Capability"])
df_cq2

,Participant,Capability
0,AI Assistant,Sensing Capability
1,AI Assistant,Reasoning Capability
2,AI Assistant,Explanation Capability
3,Human User,Self-Reporting Capability


#### 🤖 CQ 3 🤖

**Given an agent's capabilities, which tasks and goals can the agent be assigned to because it satisfies all required capabilities?**

In [ ]:
cq3 = """
PREFIX hi: <https://w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?goalLabel ?taskLabel
WHERE {
    VALUES ?capability { hi-s1:CapabilitySensing hi-s1:CapabilityReasoning hi-s1:CapabilityExplanation }
    
    ?scenario       hi:describesUseCase     ?usecase .
    ?usecase        hi:introducesHITeam     ?team .
    ?team           hi:hasGoal              ?goal ;
                    hi:hasMember            ?participant .
    ?goal           rdfs:label              ?goalLabel ;
                    hi:requiresTask         ?task .
    ?task           rdfs:label              ?taskLabel .
    ?participant    hi:hasCapability        ?capability .

}
"""

results_cq3 = g.query(cq3)
df_cq3 = pd.DataFrame([(str(row.goalLabel), str(row.taskLabel)) for row in results_cq3],
                        columns=["Goal", "Task"])
df_cq3

,Goal,Task
0,User Wellbeing Goal.,Data Collection Task
1,User Wellbeing Goal.,Conflict Resolution Task
2,User Wellbeing Goal.,Explanation Task


#### 🤖 CQ 4 🤖

**What are the constraints influencing a given goal?**

In [ ]:
cq4 = """
PREFIX hi: <https://w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?constraintLabel ?constraintComment
WHERE {
    VALUES ?goal { hi-s1:GoalUserWellbeing }
    
    ?team           hi:hasGoal              ?goal ;
                    hi:hasEthicalConstraint ?constraint .
    ?goal           rdfs:label              ?goalLabel .
    ?constraint     rdfs:label              ?constraintLabel ;
                    rdfs:comment            ?constraintComment .
}
"""

results_cq4 = g.query(cq4)
df_cq4 = pd.DataFrame([(str(row.constraintLabel), str(row.constraintComment)) for row in results_cq4],
                        columns=["Constraint Label ", "Constraint Comment"])
df_cq4

,Constraint Label,Constraint Comment
0,Privacy Constraint.,User data must be collected and processed resp...
1,Autonomy Constraint.,The assistant must respect user autonomy in li...
2,Transparency Constraint.,The assistant must explain reasoning processes...


#### 🤖 CQ 5 🤖

**Which methods are used by agents to carry out a given task, and what are their intended goals?**

In [ ]:
cq5 = """
PREFIX hi: <https://w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?methodLabel ?methodComment ?goalLabel ?goalComment
WHERE {
    VALUES ?task { hi-s1:TaskConflictResolution }
    
    ?taskExecution  hi:realizesTask         ?task ;
                    hi:usesMethod           ?method ;
                    hi:towardsGoal          ?goal .
    ?method         rdfs:label              ?methodLabel ;
                    rdfs:comment            ?methodComment .
    ?goal           rdfs:label              ?goalLabel ;
                    rdfs:comment            ?goalComment .             

}
"""

results_cq5 = g.query(cq5)
df_cq5 = pd.DataFrame([(str(row.methodLabel), str(row.methodComment), str(row.goalLabel), str(row.goalComment)) for row in results_cq5],
                        columns=["Method Label ", "Method Comment", "Goal Label ", "Goal Comment"])
df_cq5

,Method Label,Method Comment,Goal Label,Goal Comment
0,Argumentation-based Reasoning Method.,Argumentation-based reasoning used by the assi...,User Wellbeing Goal.,Maintain and improve user health habits throug...


#### 🤖 CQ 6 🤖

**What evaluations, metrics, and experiments have been conducted to assess the performance of a given Hybrid Intelligence team?**

In [ ]:
cq6 = """
PREFIX hi: <https://w3id.org/hi-ontology#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?evaluationLabel ?evaluationComment ?methodLabel ?methodComment ?goalLabel ?goalComment
WHERE {
    VALUES ?team { hi-s1:HITeamPersonalAssistant }

    ?team           hi:operatesInContext    ?context .
    ?context        hi:influences           ?evaluation .
    ?evaluation     a                       hi:Evaluation ;   
                    rdfs:label              ?evaluationLabel ;
                    rdfs:comment            ?evaluationComment ;
                    hi:usesMetric           ?metric ;
                    hi:hasExperiment        ?experiment .
    ?metric         rdfs:label              ?methodLabel ;
                    rdfs:comment            ?methodComment .
    ?experiment     rdfs:label              ?goalLabel ;
                    rdfs:comment            ?goalComment .  

}
"""

results_cq6 = g.query(cq6)
df_cq6 = pd.DataFrame([(str(row.evaluationLabel), str(row.evaluationComment), 
                        str(row.methodLabel), str(row.methodComment), 
                        str(row.goalLabel), str(row.goalComment)) for row in results_cq6],
                        columns=["Evaluation Label", "Evaluation Comment", "Method Label ", "Method Comment", "Goal Label ", "Goal Comment"])
df_cq6

,Evaluation Label,Evaluation Comment,Method Label,Method Comment,Goal Label,Goal Comment
0,Evaluation of Transparency Impact on User Trust.,Evaluates how providing explanations affects u...,Trust Preservation Metric.,Quantifies user trust retention when explanati...,Experiment on Transparency Impact on User Trust.,Experiment to assess how explanations affect u...


#### 🤖 CQ 7 🤖

**Can we identify when an agent’s behavior aligns or conflicts with an ethical constraint?**

#### 🤖 CQ 8 🤖

**Can we determine whether a team collectively possesses all capabilities needed to achieve a goal?**

#### 🤖 CQ 9 🤖

**Which Hybrid Intelligence scenarios share goals, tasks, or ethical considerations, allowing knowledge to be reused?**

#### 🤖 CQ 10 🤖

**Can reasoning infer implicit relations such as team eligibility, task assignment, or ethical influence that are not explicitly stated?**